# Natural disasters, Climate change and Economic development

## 1. Motivation

### 1.1 What is your dataset?

We first decided we wanted to analyze and visualize the effect of natural disasters, especially droughts and heatwaves; the dataset on the impact of natural disasters from 1900 was obtained from.... . It is known that natural disasters are getting more frequent with climate change, but we wanted to look into how natural disasters affected different countries based on how developed their economy os and their economic indicators. Therefore we also used economic and social data, such as GDP,  agriculture data, income data, and water and energy use. We also used population data to normalize and visualize our data. The economic and social data came from the world bank.

### 1.2 Why did you choose this/these particular dataset(s)?

The dataset about the effect of disasters from 1900 was our main interest, motivation and gave us the idea for the project. The population data was used to normalize and in some of the visualizations. We chose the specific economic indicators because .... 

### 1.3 What was your goal for the end user´s experience?

We wanted to tell the story of how natural disasters do not discriminate. Instead, they exacerbate existing inequalities, hitting countries with limited economic resources the hardest in an interactive and exploratory way. Give the reader a better insight into how natural disasters affect less economically developed countries and that the impact of climate change is also a server equity case. 

## 2. Basic stats 

We begin by importing all of the packages needed for the project

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import os
import matplotlib.pyplot as plt
import calplot
from pandas.api.types import CategoricalDtype
import seaborn as sns
import plotly.express as px
import folium
import chardet
import geopandas as gpd
from geopandas.tools import geocode
import warnings
from geopy.geocoders import Nominatim

In [2]:
import plotly.express as px
import plotly.graph_objs as go

### 2.1 Data preperation and cleaning 
#### *Write about your choices in data cleaning and preprocessing

Lets now import all the datasets 

In [3]:
# Opening file

names = ["Dis No"]

#Natural_Disasters = pd.read_csv('NaturalDisastersv3.xlsx',names=names, lineterminator='\n', nrows=25882)
Natural_Disasters = pd.read_excel('NaturalDisasters.xlsx')

In [4]:
#there are also data for technological disasters so we are choosing Natural Disasters

Natural_Disasters = Natural_Disasters[Natural_Disasters['Disaster Group']=='Natural']

#there are some natural disasters that there are very few data
Natural_Disasters = Natural_Disasters[(Natural_Disasters['Disaster Subtype']!='Landslide')& (Natural_Disasters['Disaster Subtype']!='Tsunami') & (Natural_Disasters['Disaster Subtype']!='Mudslide')&(Natural_Disasters['Disaster Subtype']!='Ground movement')&(Natural_Disasters['Disaster Subtype']!='Lava flow')& (Natural_Disasters['Disaster Subtype']!='Rockfall')& (Natural_Disasters['Disaster Subtype']!='Grasshopper')& (Natural_Disasters['Disaster Subtype']!='Locust')& (Natural_Disasters['Disaster Subtype']!='Worms')& (Natural_Disasters['Disaster Subtype']!='Subsidence')
                    & (Natural_Disasters['Disaster Subtype']!='Ash fall')]
# print( pd.unique(Natural_Disasters['Disaster Subtype'].values))
# & (Natural_Disasters['Disaster Subtype']!='Tsunami') & (Natural_Disasters['Disaster Subtype']!='Mudslide')&(Natural_Disasters['Disaster Subtype']!='Ground movement')]



#cols=["year","month","day"]
#df['date'] = df[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")


In [5]:
Countries_GDPC = pd.read_excel('GDP_per_capita.xls')

In [6]:
population = pd.read_excel('population.xls')

quite a lot of data cleaning had to take place on the data sets to be able to merge the all data together

2.1.1 GDP per captia for all of the countries in the right form, 

In [7]:
# Drop the first 3 rows of the DataFrame
Countries_GDPC = Countries_GDPC.drop([0, 1])

# Set the 4th row as the column index
Countries_GDPC.columns = Countries_GDPC.iloc[0]
Countries_GDPC = Countries_GDPC.drop(Countries_GDPC.index[0])

Countries_GDPC.index = Countries_GDPC.index.astype(int)
Countries_GDPC = Countries_GDPC.reset_index(drop=True)

Countries_GDPC = Countries_GDPC.drop('Indicator Name', axis=1)
Countries_GDPC = Countries_GDPC.drop('Indicator Code', axis=1)

Rename the year columns


In [8]:

old_names = list(Countries_GDPC.columns)
old_names = old_names[2:]


In [9]:
new_names = [str(year) for year in range(1960, 2022)]
name_map = dict(zip(old_names, new_names))

In [10]:
Countries_GDPC = Countries_GDPC.rename(columns=name_map)

In [11]:
Countries_GDPC = Countries_GDPC.drop('Country Name', axis=1)

Now we convert the dataset from having the years as columns to only having one column to indicate the years

In [12]:
df_melted = Countries_GDPC.melt(id_vars=['Country Code'], var_name='year', value_name='GDPC')
df_melted= df_melted.rename(columns={'Country Code': 'ISO','year':'Year'})

Now lest do the same for the population data

In [13]:
# Drop the first 2 rows of the DataFrame
population = population.drop([0, 1])

# Set the 4th row as the column index
population.columns = population.iloc[0]
population = population.drop(population.index[0])

population.index = population.index.astype(int)
population = population.reset_index(drop=True)
population = population.drop('Indicator Name', axis=1)
population = population.drop('Indicator Code', axis=1)

In [14]:
old_names = list(population.columns)
old_names = old_names[2:]

In [15]:
new_names = [str(year) for year in range(1960, 2022)]
name_map = dict(zip(old_names, new_names))

In [16]:
population = population.rename(columns=name_map)
population = population.drop('Country Name', axis=1)

In [17]:
df_melted_pop = population.melt(id_vars=['Country Code'], var_name='year', value_name='population')
df_melted_pop= df_melted_pop.rename(columns={'Country Code': 'ISO','year':'Year'})

Let's now merge the natural disaster dataset with data about countries GDP per capita and population.

In [18]:
NDdf = Natural_Disasters[['Dis No', 'Country','ISO', 'Region', 'Continent','Year','Total Deaths', 'Total Affected',
       "Total Damages ('000 US$)"]]

In [19]:
#NDdf = NDdf[NDdf['Year'] >= 1973]

In [20]:
ISO_Region_Continent = NDdf[['Country','ISO', 'Region', 'Continent']]
ISO_Region_Continent = ISO_Region_Continent.groupby(['ISO']).first().reset_index()
NDdf_Country = NDdf.groupby(['ISO','Year']).sum()

In [21]:
df_melted['Year'] = df_melted['Year'].astype('int64')
df_melted_pop['Year'] = df_melted_pop['Year'].astype('int64')

In [22]:
ND_GDPC_Country = pd.merge(NDdf_Country, df_melted, on=['ISO','Year'])

In [23]:
ND_GDPC_Country = pd.merge(ND_GDPC_Country , df_melted_pop, on=['ISO','Year'])

In [24]:
ND_GDPC = pd.merge(ND_GDPC_Country, ISO_Region_Continent, on=['ISO'])

Let's make two new columns Deaths per 100.000 and Affected per 100.000

In [25]:
ND_GDPC['Deaths per 100.000'] = (ND_GDPC['Total Deaths']/ND_GDPC['population'])*1000000

In [26]:
ND_GDPC['Affected per 100.000'] = (ND_GDPC['Total Affected']/ND_GDPC['population'])*1000000

In [27]:
ND_GDPC_Continent = ND_GDPC.groupby(['Continent','Year']).agg({'Total Deaths': 'sum','Total Affected': 'sum', "Total Damages ('000 US$)": 'sum','Deaths per 100.000':'sum',
                                                               'Affected per 100.000':'sum', 'GDPC' :'mean'}) 

KeyError: "Column(s) ['Affected per 100K', 'Deaths per 100K'] do not exist"

In [ ]:
ND_GDPC_2010 = ND_GDPC[ND_GDPC['Year'] >= 2010]

In [ ]:
ND_GDPC_1995 = ND_GDPC[ND_GDPC['Year'] >= 1995]

In [ ]:
import plotly.express as px

df = px.data.gapminder()
fig = px.scatter(ND_GDPC_2010, x="GDPC", y="Affected per 100K", animation_frame="Year", animation_group="Country",
           size="population", color="Continent", hover_name="Country",
           log_x=True, log_y=True, size_max=55, range_x=[50,500000], range_y=[1,5000000])

fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.update_layout(width=900, height=700)
fig.show()

In [ ]:
df = px.data.gapminder()
fig = px.scatter(ND_GDPC_1995, x="GDPC", y="Affected per 100K", animation_frame="Year", animation_group="Country",
           size="population", color="Continent", hover_name="Country",
           log_x=True, log_y=True, size_max=55, range_x=[50,500000], range_y=[0.1,5000000])

fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.update_layout(width=900, height=700)
fig.write_html("scatter_gdpc.html")
fig.show()

## 3. Data Analysis

### 3.1 Descrive your data analysis and explain what you've learned about the dataset 

## 4. Genre

### 4.1 Which tools did you use from each of the 3 categories of *Visual Narrative* (Figure 7 in Segal and Heer). Why?

### 4.2 Which tools did you use from each of the 3 categories of *Narrative Structure* (Figure 7 in Segal and Heer). Why?

## 5. Visualization

### 5.1 Explain the visualizations you've chosen.

### 5.2 Why are they right for the story you want to tell?

## 6. Discussion. Think critically about your creation

### 6.1 What went well?

### 6.2 What is still missing? What could be improved?, Why?

## 7. Contributions. Who did what?